# SciPy Image Processing
This notebook covers denoising, edge detection, morphology, connected-component measurements, and geometric transforms with practical examples.

In [ ]:
import numpy as np
from scipy import ndimage

np.set_printoptions(precision=4, suppress=True)

## 1. Image Filtering and Denoising
Filtering suppresses random noise and improves downstream feature extraction reliability.

In [ ]:
rng = np.random.default_rng(15)

x = np.linspace(-2, 2, 128)
xx, yy = np.meshgrid(x, x)
base = np.exp(-(xx ** 2 + yy ** 2) * 2.5)
noisy = base + rng.normal(0, 0.12, size=base.shape)
gauss = ndimage.gaussian_filter(noisy, sigma=1.1)

print("Example 1: Gaussian denoising")
print(f"Noisy std: {np.std(noisy):.4f}")
print(f"Filtered std: {np.std(gauss):.4f}")

salt_pepper = noisy.copy()
mask_salt = rng.random(base.shape) < 0.02
mask_pepper = rng.random(base.shape) < 0.02
salt_pepper[mask_salt] = 1.0
salt_pepper[mask_pepper] = -1.0
median = ndimage.median_filter(salt_pepper, size=3)

print()
print("Example 2: Median filtering for impulse noise")
print(f"Extreme pixels before: {np.sum(np.abs(salt_pepper) > 0.9)}")
print(f"Extreme pixels after: {np.sum(np.abs(median) > 0.9)}")

## 2. Edge Detection
Gradient and Laplacian operators detect sharp structural transitions in images.

In [ ]:
img = np.zeros((120, 160), dtype=float)
img[20:100, 30:130] = 0.6
img[45:75, 60:100] = 1.0

sobel_x = ndimage.sobel(img, axis=1)
sobel_y = ndimage.sobel(img, axis=0)
grad_mag = np.hypot(sobel_x, sobel_y)

print("Example 1: Sobel edge magnitude")
print(f"Max gradient: {grad_mag.max():.4f}")
print(f"Edge pixels above 1.0: {np.sum(grad_mag > 1.0)}")

defect = img.copy()
defect[30:35, 85:120] -= 0.4
lap = ndimage.laplace(defect)

print()
print("Example 2: Laplacian defect response")
print(f"Laplacian min: {lap.min():.4f}")
print(f"Laplacian max: {lap.max():.4f}")

## 3. Morphological Operations
Morphology removes small artifacts, closes gaps, and fills holes in segmentation masks.

In [ ]:
rng = np.random.default_rng(99)
mask = np.zeros((80, 100), dtype=bool)
mask[20:65, 25:75] = True
noise_idx = rng.integers(0, mask.size, size=120)
mask.ravel()[noise_idx] = True

opened = ndimage.binary_opening(mask, structure=np.ones((3, 3), dtype=bool))
closed = ndimage.binary_closing(opened, structure=np.ones((5, 5), dtype=bool))

print("Example 1: Opening and closing")
print(f"Foreground before: {mask.sum()}")
print(f"Foreground after: {closed.sum()}")

organ = np.zeros((90, 90), dtype=bool)
organ[18:72, 18:72] = True
organ[40:50, 40:50] = False
filled = ndimage.binary_fill_holes(organ)

print()
print("Example 2: Hole filling")
print(f"Pixels before fill: {organ.sum()}")
print(f"Pixels after fill: {filled.sum()}")

## 4. Connected Components and Measurements
Connected-component analysis extracts counts, areas, centers, and bounding boxes for detected objects.

In [ ]:
mask = np.zeros((100, 120), dtype=bool)
mask[10:28, 14:30] = True
mask[40:70, 35:63] = True
mask[72:90, 80:110] = True

labels, n_obj = ndimage.label(mask)
areas = ndimage.sum(mask, labels=labels, index=np.arange(1, n_obj + 1))
centers = ndimage.center_of_mass(mask, labels=labels, index=np.arange(1, n_obj + 1))

print("Example 1: Object counting and area")
print(f"Objects: {n_obj}")
print("Areas:", [int(a) for a in areas])

slices = ndimage.find_objects(labels)
boxes = []
for sl in slices:
    boxes.append((sl[0].stop - sl[0].start, sl[1].stop - sl[1].start))

print()
print("Example 2: Centers and bounding boxes")
print("Centers:", [(round(c[0], 2), round(c[1], 2)) for c in centers])
print("Box shapes:", boxes)

## 5. Geometric Transformations
Geometric transforms align and normalize image orientation and scale for modeling pipelines.

In [ ]:
img = np.zeros((96, 96), dtype=float)
img[28:68, 36:60] = 1.0
img[42:54, 60:82] = 0.7

rot = ndimage.rotate(img, angle=22, reshape=False, order=1)
zoomed = ndimage.zoom(img, zoom=1.25, order=1)

print("Example 1: Rotation and zoom")
print(f"Original shape: {img.shape}")
print(f"Rotated shape: {rot.shape}")
print(f"Zoomed shape: {zoomed.shape}")

matrix = np.array([[1.0, -0.15], [0.05, 1.0]])
affine = ndimage.affine_transform(img, matrix=matrix, offset=[2, -4], order=1)

print()
print("Example 2: Affine deskew")
print(f"Intensity sum before: {img.sum():.2f}")
print(f"Intensity sum after: {affine.sum():.2f}")